<a href="https://colab.research.google.com/github/Nsimaar99/Nsima-Project/blob/master/nsiudacity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!git clone https://github.com/Nsimaar99/Nsima-Project.git


fatal: destination path 'Nsima-Project' already exists and is not an empty directory.


# Load the data

In [56]:
# Step 1: Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# PyTorch Libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
import torchvision.models as models
import torch.optim as optim
from PIL import Image
import torch.nn.functional as F

# Scikit-learn Libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Import the necessary libraries for downloading and handling data files
import requests  # For sending HTTP requests to download files
import tarfile   # For extracting .tar.gz files
import io        # For handling in-memory file-like objects
import os        # For interacting with the operating system

In [57]:
# Step 2: Download the tar.gz file (if not already downloaded)
url = "https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz"
filename = "flower_data.tar.gz"  # Name of the downloaded file
if not os.path.exists(filename):
    print("Downloading the dataset...")
    response = requests.get(url)
    response.raise_for_status()  # Ensure the request was successful
    with open(filename, 'wb') as f:
        f.write(response.content)
    print("Download complete!")
else:
    print("Dataset already downloaded.")

# Step 3: Extract the contents of the tar.gz file (if not already extracted)
extract_path = 'flowers'  # Directory to extract to
if not os.path.exists(extract_path):
    print("Extracting the dataset...")
    with tarfile.open(filename, mode='r:gz') as tar:
        tar.extractall(path=extract_path)
    print("Extraction complete!")
else:
    print("Dataset already extracted.")

# Step 4: Verify the directory structure
print(f"Files extracted to: {extract_path}")
print("Directory structure:")
for root, dirs, files in os.walk(extract_path):
    level = root.replace(extract_path, '').count(os.sep)
    indent = ' ' * 4 * (level)
    print(f"{indent}{os.path.basename(root)}/")
    for f in files:
        print(f"{indent}    {f}")


Streaming output truncated to the last 5000 lines.
            image_02422.jpg
            image_02317.jpg
            image_02372.jpg
            image_02338.jpg
            image_02356.jpg
            image_02322.jpg
            image_02325.jpg
            image_02420.jpg
            image_02399.jpg
            image_02342.jpg
            image_02424.jpg
            image_02442.jpg
            image_02401.jpg
            image_02383.jpg
            image_02438.jpg
            image_02415.jpg
            image_02404.jpg
            image_02323.jpg
            image_02332.jpg
            image_02343.jpg
            image_02390.jpg
            image_02416.jpg
            image_02360.jpg
            image_02337.jpg
            image_02324.jpg
            image_02354.jpg
            image_02333.jpg
            image_02406.jpg
            image_02395.jpg
            image_02373.jpg
            image_02340.jpg
            image_02436.jpg
            image_02410.jpg
            image_02319.j

In [58]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [60]:
# TODO: Define your transforms for the training, validation, and testing sets
# Define your transforms for the training, validation, and testing sets
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load the datasets with ImageFolder
image_datasets = {
    'train': datasets.ImageFolder(root=train_dir, transform=data_transforms['train']),
    'val': datasets.ImageFolder(root=valid_dir, transform=data_transforms['val']),
    'test': datasets.ImageFolder(root=test_dir, transform=data_transforms['test'])
}

# Define the dataloaders
dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=32, shuffle=True, num_workers=2),
    'val': DataLoader(image_datasets['val'], batch_size=32, shuffle=False, num_workers=2),
    'test': DataLoader(image_datasets['test'], batch_size=32, shuffle=False, num_workers=2)
}

# Access the number of classes
num_classes = len(image_datasets['train'].class_to_idx)
print(f"The dataset has {num_classes} classes.")


The dataset has 102 classes.


#### Label mapping
You'll also need to load in a mapping from category label to category name. You can find this in the file cat_to_name.json. It's a JSON object which you can read in with the json module. This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [61]:
import json
import os

# Define the file path
file_path = 'cat_to_name.json'

# Check if the file exists
if not os.path.exists(file_path):
    # If file is not found, prompt the user to upload it
    from google.colab import files
    uploaded = files.upload()  # This will prompt you to upload the file from your local system

    # Ensure the file was uploaded
    if file_path not in uploaded:
        raise FileNotFoundError(f"Expected file '{file_path}' not found in the uploaded files.")
else:
    print("File found.")

File found.


#### Building and training the classifier

In [ ]:
# Load a pre-trained network (If you need a starting point, the VGG networks work great and are straightforward to use)
# Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
# Train the classifier layers using backpropagation using the pre-trained network to get the features
# Track the loss and accuracy on the validation set to determine the best hyperparameters

In [62]:
# Load a pre-trained network (If you need a starting point, the VGG networks work great and are straightforward to use)
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load a pre-trained VGG model
model = models.vgg16(pretrained=True)

# Modify the final layer for a different number of classes
num_classes = 102
model.classifier[6] = nn.Linear(in_features=4096, out_features=num_classes)

# Freeze the feature parameters so we don't backpropagate through them
for param in model.parameters():
    param.requires_grad = False

# Move the model to GPU (if available)
model.to(device)

# Print the modified model architecture
print(model)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [63]:
# Feature extractor
def extract_features(model, dataloader):
    features = []
    with torch.no_grad():
        model.eval()
        for inputs, _ in dataloader:
            inputs = inputs.to(device)
            output = model.features(inputs)
            features.append(output.view(output.size(0), -1))  # Flatten the output
    return torch.cat(features)

# Example DataLoader for demonstration (replace with your actual data loader)
# dataloader = DataLoader(...)

# Extract features
# features = extract_features(vgg_model, dataloader)

In [64]:
# Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
class FeedForwardClassifier(nn.Module):
    def __init__(self, input_size, hidden_layers, output_size, dropout_prob=0.5):
        super(FeedForwardClassifier, self).__init__()
        self.dropout_prob = dropout_prob

        # Create a list of layers
        layers = []
        prev_size = input_size

        # Add hidden layers
        for hidden_size in hidden_layers:
            layers.append(nn.Linear(prev_size, hidden_size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(p=dropout_prob))
            prev_size = hidden_size

        # Add output layer
        layers.append(nn.Linear(prev_size, output_size))

        # Combine layers into a sequential model
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        x = self.network(x)
        return F.softmax(x, dim=1)  # Apply softmax to the output layer

# Initialize the feed-forward classifier
input_size = 25088  # Size of the flattened features from VGG16
hidden_layers = [512, 256, 128]  # Example sizes of hidden layers
output_size = 102  # Number of classes
dropout_prob = 0.5  # Dropout probability

classifier = FeedForwardClassifier(input_size, hidden_layers, output_size, dropout_prob)

# Move the classifier to GPU (if available)
classifier.to(device)

print(classifier)

FeedForwardClassifier(
  (network): Sequential(
    (0): Linear(in_features=25088, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=256, out_features=128, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.5, inplace=False)
    (9): Linear(in_features=128, out_features=102, bias=True)
  )
)


In [65]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(classifier.parameters(), lr=0.01, momentum=0.9)
# Move the model to GPU (if available)
classifier.to(device)

FeedForwardClassifier(
  (network): Sequential(
    (0): Linear(in_features=25088, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=256, out_features=128, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.5, inplace=False)
    (9): Linear(in_features=128, out_features=102, bias=True)
  )
)

In [1]:
# Train the classifier layers using backpropagation using the pre-trained network to get the features
# Training function with validation
def train_classifier(classifier, train_loader, validation_loader, criterion, optimizer, device, epochs=10):
    for epoch in range(epochs):
        classifier.train()  # Set the model to training mode
        running_loss = 0.0

        # Training loop
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()  # Zero the gradients
            outputs = classifier(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Calculate loss
            loss.backward()  # Backward pass
            optimizer.step()  # Update weights

            running_loss += loss.item()  # Accumulate loss

        avg_loss = running_loss / len(train_loader)  # Average loss for the epoch
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}')

        # Validation
        validate_model(classifier, validation_loader, criterion, device)

def validate_model(model, validation_loader, criterion, device):
    model.eval()  # Set the model to evaluation mode
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # No need to compute gradients during validation
        for inputs, labels in validation_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Calculate the loss
            running_loss += loss.item()

            # Get predictions
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = running_loss / len(validation_loader)
    accuracy = correct / total

    print(f'Validation Loss: {avg_loss:.4f}, Validation Accuracy: {accuracy:.4f}')



In [67]:
# TODO: Save the checkpoint
# Save the checkpoint for the classifier
torch.save(classifier.state_dict(), 'classifier_checkpoint.pth')

# Load the state dictionary back to print keys (optional)
loaded_state_dict = torch.load('classifier_checkpoint.pth')
print(loaded_state_dict.keys())

odict_keys(['network.0.weight', 'network.0.bias', 'network.3.weight', 'network.3.bias', 'network.6.weight', 'network.6.bias', 'network.9.weight', 'network.9.bias'])


<ipython-input-67-c111f78d58b9>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_state_dict = torch.load('classifier_checkpoint.pth')


In [68]:
# Define the load_model function
def load_model(checkpoint_path, input_size, hidden_layers, output_size, dropout_prob, device):
    model = FeedForwardClassifier(input_size, hidden_layers, output_size, dropout_prob)
    model.load_state_dict(torch.load(checkpoint_path))
    model.to(device)

In [69]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns a Numpy array
    '''
    # Resize the image so the shortest side is 256 pixels
    size = 256
    image.thumbnail((size, size))  # Maintain aspect ratio

    # Crop the center 224x224 of the image
    left = (image.width - 224) / 2
    top = (image.height - 224) / 2
    right = (image.width + 224) / 2
    bottom = (image.height + 224) / 2
    image = image.crop((left, top, right, bottom))

    # Convert the image to a numpy array and scale to [0, 1]
    np_image = np.array(image) / 255.0  # Convert to float [0, 1]

    # Normalize the image
    means = np.array([0.485, 0.456, 0.406])
    stds = np.array([0.229, 0.224, 0.225])
    np_image = (np_image - means) / stds

    # Rearrange dimensions to (C, H, W)
    np_image = np_image.transpose((2, 0, 1))  # Change from (H, W, C) to (C, H, W)

    return np_image

In [73]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()

    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))

    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean

    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)

    ax.imshow(image)

    return ax

In [76]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model. '''


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\TIMOTHY EMMANUEL\\Downloads\\cat_to_name.json'